In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split 
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.svm import SVR
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
#from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import r2_score


In [2]:
Crop_ds=pd.read_csv("crop_preprocess_ds.csv")
Crop_ds

,Crop_Year,Area_Hectares,Production_Tonnes,Rainfall_mm,Fertilizer_Kgms,Pesticide_Kgms,Yield/unitArea,Crop_Arhar/Tur,Crop_Bajra,Crop_Banana,...,State_Odisha,State_Puducherry,State_Punjab,State_Sikkim,State_Tamil Nadu,State_Telangana,State_Tripura,State_Uttar Pradesh,State_Uttarakhand,State_West Bengal
0,1997,73814.0,56708,2051.4,7024878.38,22882.34,0.796087,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1997,6637.0,4685,2051.4,631643.29,2057.47,0.710435,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1997,796.0,22,2051.4,75755.32,246.76,0.238333,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1997,19656.0,309194,2051.4,1870661.52,6093.36,5.118226,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1997,1739.0,794,2051.4,165500.63,539.09,0.420909,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19568,1998,4000.0,2000,1498.0,395200.00,1160.00,0.500000,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19569,1998,1000.0,3000,1498.0,98800.00,290.00,3.000000,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19570,1997,187823.5,309194,1356.2,25054187.10,49993.35,1.285000,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19571,1997,187823.5,5488,1356.2,25054187.10,49993.35,0.016667,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
Crop_ds.isnull().sum()

Crop_Year              0
Area_Hectares          0
Production_Tonnes      0
Rainfall_mm            0
Fertilizer_Kgms        0
                      ..
State_Telangana        0
State_Tripura          0
State_Uttar Pradesh    0
State_Uttarakhand      0
State_West Bengal      0
Length: 95, dtype: int64

In [4]:
Crop_ds.dropna(inplace=True)

In [5]:
indep_X=Crop_ds.drop('Yield/unitArea',axis=1)
dep_Y = Crop_ds[['Yield/unitArea']]
dep_Y

,Yield/unitArea
0,0.796087
1,0.710435
2,0.238333
3,5.118226
4,0.420909
...,...
19568,0.500000
19569,3.000000
19570,1.285000
19571,0.016667


In [6]:
indep_X.shape

(19573, 94)

In [7]:
dep_Y.shape

(19573, 1)

In [9]:
def split_scalar(indep_X,dep_Y):
        from sklearn.model_selection import train_test_split
        X_train, X_test, y_train, y_test = train_test_split(indep_X, dep_Y, test_size = 0.50, random_state = 0)
        sc = StandardScaler()
        X_train = sc.fit_transform(X_train)
        X_test = sc.transform(X_test)    
        y_train = y_train.iloc[:len(X_train)]
        return X_train, X_test, y_train, y_test
def compute_PCA(X_train,X_test,n):
        if n>X_train.shape[0]:
            raise ValueError("Number of components cannot exceed the number of samples in the dataset.")
        pca = PCA(n_components = n)
        X_train_PCA= pca.fit_transform(X_train)
        X_test_PCA=pca.transform(X_test.iloc[:X_train_PCA.shape[0]])
        explained_variance = pca.explained_variance_ratio_
        return pca,X_train_PCA,X_test_PCA,explained_variance
def r2_prediction(regressor,X_test,y_test):
         y_pred = regressor.predict(X_test)
         from sklearn.metrics import r2_score
         r2=r2_score(y_test,y_pred)
         return r2
def svm_linear(X_train,y_train,X_test):
        from sklearn.svm import SVC
        regressor = SVC(kernel = 'linear', random_state = 0)
        regressor.fit(X_train, y_train)
        r2=r2_prediction(regressor,X_test,y_test)
        return r2
def svm_NL(X_train,y_train,X_test):
        from sklearn.svm import SVC
        regressor = SVC(kernel = 'rbf', random_state = 0)
        regressor.fit(X_train, y_train)
        r2=r2_prediction(regressor,X_test,y_test)
        return r2  
def Decision(X_train,y_train,X_test):
         # Fitting K-NN to the Training set
        from sklearn.tree import DecisionTreeRegressor
        regressor = DecisionTreeRegressor(criterion = 'entropy', random_state = 0)
        regressor.fit(X_train, y_train)
        r2=r2_prediction(regressor,X_test,y_test)
        return r2  
def random(X_train,y_train,X_test):
        # Fitting K-NN to the Training set
        from sklearn.ensemble import RandomForestRegressor
        regressor = RandomForestRegressor(n_estimators = 10, criterion = 'entropy', random_state = 0)
        regressor.fit(X_train, y_train)
        r2=r2_prediction(regressor,X_test,y_test)
        return r2
def PCA_ML(accsvml,accsvmnl,accdes,accrf):
    dataframe=pd.DataFrame(index=['PCA n=6'],columns=['SVML','SVMNL','Decision','Random'])
    for number,idex in enumerate(dataframe.index):
        dataframe['SVML'][idex]=accsvml[number]
        dataframe['SVMNL'][idex]=accsvmnl[number]
        dataframe['Decision'][idex]=accdes[number]
        dataframe['Random'][idex]=accrf[number]
    return dataframe    

    

In [15]:

X_train, X_test, y_train, y_test = split_scalar(indep_X, dep_Y)
n_components=5
if n_components>X_train.shape[0]:
        print("Number of components exceed the number of samples in the dataset.")
else:
        pca,explained_variance,X_train_PCA,X_test_PCA=compute_PCA(X_train,X_test,n_components)

accsvml = []
accsvmnl = []
accdes = []
accrf = []
for i in range(10):
    
    r2_sl = svm_linear(X_train_PCA, y_train, X_test_PCA,y_test)
    accsvml.append(r2_sl)
    
    r2_NL = svm_NL(X_train_PCA,y_train, X_test_PCA,y_test)  
    accsvmnl.append(r2_NL)
    
    r2_d = Decision(X_train_PCA, y_train, X_test_PCA,y_test)  
    accdes.append(r2_d)
    
    r2_r = random(X_train_PCA, y_train, X_test_PCA,y_test)
    accrf.append(r2_r)

    result = PCA_ML(accsvml, accsvmnl, accdes, accrf)
    print(result)

AttributeError: 'numpy.ndarray' object has no attribute 'iloc'